In [1]:
from sklearn.datasets import load_iris
dataset = load_iris()
X = dataset.data
y = dataset.target
print(y)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]


In [2]:
attribute_means = X.mean(axis=0)
print(attribute_means)

[5.84333333 3.05733333 3.758      1.19933333]


In [3]:
import numpy as np
X_d = np.array(X >= attribute_means ,dtype = 'int')

[[0 1 0 0]
 [0 0 0 0]
 [0 1 0 0]
 [0 1 0 0]
 [0 1 0 0]
 [0 1 0 0]
 [0 1 0 0]
 [0 1 0 0]
 [0 0 0 0]
 [0 1 0 0]
 [0 1 0 0]
 [0 1 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 1 0 0]
 [0 1 0 0]
 [0 1 0 0]
 [0 1 0 0]
 [0 1 0 0]
 [0 1 0 0]
 [0 1 0 0]
 [0 1 0 0]
 [0 1 0 0]
 [0 1 0 0]
 [0 1 0 0]
 [0 0 0 0]
 [0 1 0 0]
 [0 1 0 0]
 [0 1 0 0]
 [0 1 0 0]
 [0 1 0 0]
 [0 1 0 0]
 [0 1 0 0]
 [0 1 0 0]
 [0 1 0 0]
 [0 1 0 0]
 [0 1 0 0]
 [0 1 0 0]
 [0 0 0 0]
 [0 1 0 0]
 [0 1 0 0]
 [0 0 0 0]
 [0 1 0 0]
 [0 1 0 0]
 [0 1 0 0]
 [0 0 0 0]
 [0 1 0 0]
 [0 1 0 0]
 [0 1 0 0]
 [0 1 0 0]
 [1 1 1 1]
 [1 1 1 1]
 [1 1 1 1]
 [0 0 1 1]
 [1 0 1 1]
 [0 0 1 1]
 [1 1 1 1]
 [0 0 0 0]
 [1 0 1 1]
 [0 0 1 1]
 [0 0 0 0]
 [1 0 1 1]
 [1 0 1 0]
 [1 0 1 1]
 [0 0 0 1]
 [1 1 1 1]
 [0 0 1 1]
 [0 0 1 0]
 [1 0 1 1]
 [0 0 1 0]
 [1 1 1 1]
 [1 0 1 1]
 [1 0 1 1]
 [1 0 1 1]
 [1 0 1 1]
 [1 0 1 1]
 [1 0 1 1]
 [1 0 1 1]
 [1 0 1 1]
 [0 0 0 0]
 [0 0 1 0]
 [0 0 0 0]
 [0 0 1 1]
 [1 0 1 1]
 [0 0 1 1]
 [1 1 1 1]
 [1 1 1 1]
 [1 0 1 1]
 [0 0 1 1]
 [0 0 1 1]
 [0 0 1 1]

In [4]:
from collections import defaultdict
from operator import itemgetter
def train_feature_value(X, y_true, feature_index, value): ##数据集、类别数组、选好的特征索引值、特征值
    class_counts = defaultdict(int)
    for sample, y in zip(X, y_true):
        if sample[feature_index] == value:
            class_counts[y] += 1
    sorted_class_counts = sorted(class_counts.items(), key=itemgetter(1), reverse=True)
    most_frequent_class = sorted_class_counts[0][0]
    incorrect_predictions = [class_count 
                             for class_value, class_count in class_counts.items() 
                             if class_value != most_frequent_class]
    error = sum(incorrect_predictions)
    return most_frequent_class, error

In [5]:
def train_on_feature(X, y_true, feature_index):
    values = set(X[:,feature_index]) # 以数组的 形式返回由feature_index所指的列。
    predictors = {} # 再创建字典predictors，用作预测器。字典的键为特征值，值为类别。
    errors = []
    for current_value in values:
        most_frequent_class, error = train_feature_value(X,y_true, feature_index, current_value) 
        predictors[current_value] = most_frequent_class 
        errors.append(error)
    total_error = sum(errors)
    return predictors, total_error

In [6]:
from sklearn.model_selection import train_test_split
Xd_train, Xd_test, y_train, y_test = train_test_split(X_d, y, random_state=14)
all_predictors = {}
errors = {}
for feature_index in range(Xd_train.shape[1]):
    predictors, total_error = train_on_feature(Xd_train, y_train, feature_index)
    all_predictors[feature_index] = predictors
    errors[feature_index] = total_error
best_feature, best_error = sorted(errors.items(), key=itemgetter(1))[0] # 然后找出错误率最低的特征，作为分类的唯一规则。
model = {'feature':best_feature,'predictor':all_predictors[best_feature][0]} #model模型是一个字典结构，包含两个元素:用于分类的特征和预测器。

In [7]:
def predict(X_test, model):
    variable = model['feature']
    predictor = model['predictor']
    y_predicted = np.array([predictor[int(sample[variable])] for sample in X_test])
    return y_predicted

In [8]:
y_predicted = predict(Xd_test, model)
print(y_predicted)

IndexError: invalid index to scalar variable.

In [ ]:
accuracy = np.mean(y_predicted == y_test) * 100
print("The test accuracy is {:.1f}%".format(accuracy))